# Imports

In [336]:
import pandas as pd

import re

from time import sleep

from geopy.geocoders import GoogleV3

# Data

In [337]:
stores = pd.read_csv('../data/store_list_v2_clean.csv')
products = pd.read_csv('../data/products_tableau.csv')
profits = pd.read_csv('../data/profits_tableau.csv')

# Cleaning Store Data

## Stores

In [338]:
stores.drop(columns=['ID'], inplace=True)
stores.head()

,Store,Address,City,State
0,Berkeley Bowl,2020 Oregon St,Berkeley,California
1,Berkeley Bowl West,920 Heinz Avenue,Berkeley,California
2,Diablo Foods-Lafeyette,3615 Mt Diablo Blvd,Lafayette,California
3,Draeger's-Danville,291 Utah Ave.,South San Francisco,California
4,Encinal Market,3211 Encinal Ave.,Alameda,California


Check for missing values

In [339]:
stores.isna().sum()

Store      0
Address    1
City       0
State      0
dtype: int64

In [341]:
stores[stores['Address'].isna()]

,Store,Address,City,State
291,Club Costa Vida/WFSH,NaN,Santa Rosa,California


Dropping this store because there is no address data that could be fine. Only one order in profits data

In [342]:
stores = stores[stores['Address'] != 'Club Costa Vida/WFSH']

Check for stores that have a P.O. box as the address. These will be fixed manually. 

In [343]:
stores[stores['Address'].str.contains('P.O.', na=False)]

,Store,Address,City,State
30,The Tides Bait & Tackle,P.O. Box 518,Bodega Bay,California
31,The Tides Unocal,P.O. Box 518,Bodega Bay,California


Combine street address, city, and state to a full address.

In [344]:
stores['Full Address'] = stores['Address'] + ', ' + stores['City'] + ', ' + stores['State']

In [345]:
stores.head()

,Store,Address,City,State,Full Address
0,Berkeley Bowl,2020 Oregon St,Berkeley,California,"2020 Oregon St, Berkeley, California"
1,Berkeley Bowl West,920 Heinz Avenue,Berkeley,California,"920 Heinz Avenue, Berkeley, California"
2,Diablo Foods-Lafeyette,3615 Mt Diablo Blvd,Lafayette,California,"3615 Mt Diablo Blvd, Lafayette, California"
3,Draeger's-Danville,291 Utah Ave.,South San Francisco,California,"291 Utah Ave., South San Francisco, California"
4,Encinal Market,3211 Encinal Ave.,Alameda,California,"3211 Encinal Ave., Alameda, California"


Convert to a list on dictionaries to iterate over.

In [346]:
stores_list = stores.to_dict(orient='records')
stores_list[:2]

[{'Store': 'Berkeley Bowl',
  'Address': '2020 Oregon St',
  'City': 'Berkeley',
  'State': 'California',
  'Full Address': '2020 Oregon St, Berkeley, California'},
 {'Store': 'Berkeley Bowl West',
  'Address': '920 Heinz Avenue',
  'City': 'Berkeley',
  'State': 'California',
  'Full Address': '920 Heinz Avenue, Berkeley, California'}]

In [347]:
stores_list[0]['Full Address']

'2020 Oregon St, Berkeley, California'

In [16]:
geolocator = GoogleV3(api_key='')
location = geolocator.geocode('12830 Valley Ford Rd., Petaluma, California')
print((location.latitude, location.longitude))

(38.3135992, -122.8759106)


Instantiate an empty list. Failed queries will be added to it to examine and fix manually.

In [18]:
failed_lst = []

Iterates over all the stores in the list. Uses geopy, GoogleV3 and the full address to find the latitude and longitude. These coordinates will be needed for the Tableau dashboard.

In [19]:
# for store in stores_list:
    
#     try:
#         location = geolocator.geocode(store['Full Address'])

#         store['lat'] = location.latitude
#         store['lon'] = location.longitude
#         print(f"ADDED: {store['Full Address']}") 
    
#     except:
#         failed_lst.append(store['Full Address'])
#         print(f"FAILED: {store['Full Address']}")

#     sleep(1)

ADDED: 2020 Oregon St, Berkeley, California
ADDED: 920 Heinz Avenue, Berkeley, California
ADDED: 3615 Mt Diablo Blvd, Lafayette, California
ADDED: 291 Utah Ave., South San Francisco, California
ADDED: 3211 Encinal Ave., Alameda, California
ADDED: 3420 Fruitvale Ave., Oakland, California
ADDED: 3501 MacArthur Blvd, Oakland, California
ADDED: 7000 Bancroft Avenue, Oakland, California
ADDED: 820 Sycamore Valley Rd., Danville, California
ADDED: 5804 College Ave., Oakland, California
ADDED: 3121 Castro Valley Blvd., Castro Valley, California
ADDED: 2082 San Pablo Ave., Berkeley, California
ADDED: 1582 Hopkins St., Berkeley, California
ADDED: 1550 Hopkins Street, Berkeley, California
ADDED: 160 Washington Ave., Richmond, California
ADDED: 6310 College Avenue, Oakland, California
ADDED: 155 W. Richmond Ave., Richmond, California
ADDED: 5885 Broadway Terrace, Oakland, California
ADDED: 2390 Powell St., Emeryville, California
ADDED: 12830 Valley Ford Rd., Petaluma, California
ADDED: 1275 Hwy. 1

In [348]:
len(failed_lst)

6

Coordinates will be retrieved for these addresses manually.

In [349]:
failed_lst

['75 Loch Lomond Drive, San Rafael, California',
 '100 Red Hill Ave, San Anselmo, California',
 '342 1st St, Los Altos, California',
 '500 Grove Street, San Francisco, California',
 '4751 Geary Blvd., San Francisco, California',
 '546 E Cotati Ave, Cotati, California']

Save stores to csv to edit manually

In [350]:
pd.DataFrame(stores_list).to_csv('../data/store_coords.csv', index=False)

Read in completed store coords file

In [396]:
store_coords = pd.read_csv('../data/store_coords_final.csv')
store_coords.head()

,Store,Address,City,State,Full Address,lat,lon
0,Berkeley Bowl,2020 Oregon St,Berkeley,California,"2020 Oregon St, Berkeley, California",37.857113,-122.267250
1,Berkeley Bowl West,920 Heinz Avenue,Berkeley,California,"920 Heinz Avenue, Berkeley, California",37.853534,-122.290376
2,Diablo Foods-Lafeyette,3615 Mt Diablo Blvd,Lafayette,California,"3615 Mt Diablo Blvd, Lafayette, California",37.890698,-122.124482
3,Draeger's-Danville,291 Utah Ave.,South San Francisco,California,"291 Utah Ave., South San Francisco, California",37.647738,-122.394870
4,Encinal Market,3211 Encinal Ave.,Alameda,California,"3211 Encinal Ave., Alameda, California",37.756290,-122.233672


In [397]:
store_coords.isna().sum()

Store           0
Address         0
City            0
State           0
Full Address    0
lat             0
lon             0
dtype: int64

## Products

In [398]:
products.head()

,territory,sold_to_name,item_description,item_type,quantity,price,date
0,EB,Monterey Foods Market,Have'a Corn Chips,Chips,48.0,76.80,2017-01-01
1,SF,Madera Liquor,Have'a Corn Chips,Chips,12.0,19.20,2017-01-01
2,MR,Mollie Stone #1-Sausalito,Casa Linda Enchilada Sauce,Sauce,12.0,42.12,2017-01-01
3,SF,Fulton & Brodrick,Loacker Kakao Cookies,Cookie,4.0,14.28,2017-01-01
4,MR,Mollie Stone #1-Sausalito,Andy's Rub - 7 oz,Spice,12.0,64.92,2017-01-01


Fixing store names

In [399]:
products['sold_to_name'] = products['sold_to_name'].str.replace('Harvest Urban Market-8th St.', 'Harvest Urban Mkt-8th St.')
products['sold_to_name'] = products['sold_to_name'].str.replace('Zanottos Family Mkt - Naglee', 'Zanottos - Naglee')
products['sold_to_name'] = products['sold_to_name'].str.replace("Guigni's Deli", "Giugni's Grocery")

C:\Users\ncox0\AppData\Local\Temp/ipykernel_3044/2583582292.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  products['sold_to_name'] = products['sold_to_name'].str.replace('Harvest Urban Market-8th St.', 'Harvest Urban Mkt-8th St.')


Merging products and store addresses

In [400]:
df_products = pd.merge(
    products, 
    store_coords, 
    left_on='sold_to_name',
    right_on='Store',
    how='left'
)

df_products.head()

,territory,sold_to_name,item_description,item_type,quantity,price,date,Store,Address,City,State,Full Address,lat,lon
0,EB,Monterey Foods Market,Have'a Corn Chips,Chips,48.0,76.80,2017-01-01,Monterey Foods Market,1550 Hopkins Street,Berkeley,California,"1550 Hopkins Street, Berkeley, California",37.881133,-122.282269
1,SF,Madera Liquor,Have'a Corn Chips,Chips,12.0,19.20,2017-01-01,Madera Liquor,4401 Geary Blvd.,San Francisco,California,"4401 Geary Blvd., San Francisco, California",37.780738,-122.466572
2,MR,Mollie Stone #1-Sausalito,Casa Linda Enchilada Sauce,Sauce,12.0,42.12,2017-01-01,Mollie Stone #1-Sausalito,100 Harbor Dr.,Sausalito,California,"100 Harbor Dr., Sausalito, California",37.866654,-122.499525
3,SF,Fulton & Brodrick,Loacker Kakao Cookies,Cookie,4.0,14.28,2017-01-01,Fulton & Brodrick,667 Broderick St.,San Francisco,California,"667 Broderick St., San Francisco, California",37.776500,-122.440101
4,MR,Mollie Stone #1-Sausalito,Andy's Rub - 7 oz,Spice,12.0,64.92,2017-01-01,Mollie Stone #1-Sausalito,100 Harbor Dr.,Sausalito,California,"100 Harbor Dr., Sausalito, California",37.866654,-122.499525


In [401]:
df_products.drop_duplicates(inplace=True)

In [402]:
len(products), len(df_products)

(255305, 255381)

In [403]:
df_products.isna().sum()

territory           0
sold_to_name        0
item_description    0
item_type           0
quantity            0
price               0
date                0
Store               0
Address             0
City                0
State               0
Full Address        0
lat                 0
lon                 0
dtype: int64

In [404]:
df_products.to_csv('../data/products_locations.csv', index=False)

## Profits

In [405]:
profits.head()

,sold_to_name,product_line,quantity,gross_profit,territory,date
0,Farmer Joe's - Fruitvale Ave.,DCD,1941.0,2006.37,EB,2017-01-01
1,"Seven Eleven-Redwood Hwy,M.V.",DCD,120.0,37.92,MR,2017-01-01
2,Nugget - #15 Corte Madera,DCD,144.0,32.85,MR,2017-01-01
3,Nugget - #15 Corte Madera,DCD,57.0,45.41,MR,2017-01-01
4,Nugget - #16 Novato,DCD,165.0,53.76,MR,2017-01-01


In [406]:
profits['sold_to_name'] = profits['sold_to_name'].str.replace('Harvest Urban Market-8th St.', 'Harvest Urban Mkt-8th St.')
profits['sold_to_name'] = profits['sold_to_name'].str.replace('Zanottos Family Mkt - Naglee', 'Zanottos - Naglee')
profits['sold_to_name'] = profits['sold_to_name'].str.replace("Guigni's Deli", "Giugni's Grocery")

profits['sold_to_name'] = profits['sold_to_name'].str.replace('Safeway #2718-Mill Valley', 'Safeway #2718-Strawberry')
profits['sold_to_name'] = profits['sold_to_name'].str.replace('Safeway #2781-Mill Valley', 'Safeway #2718-Strawberry')

profits['sold_to_name'] = profits['sold_to_name'].str.replace('Safeway #979 Novato', 'Safeway #979-Diablo Ave,Nov.')

C:\Users\ncox0\AppData\Local\Temp/ipykernel_3044/116638897.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  profits['sold_to_name'] = profits['sold_to_name'].str.replace('Harvest Urban Market-8th St.', 'Harvest Urban Mkt-8th St.')


In [407]:
profits['sold_to_name'] = profits['sold_to_name'].str.replace('76 Stn - Ignacio', 'Ignacio Gas/76 Stn')
profits['sold_to_name'] = profits['sold_to_name'].str.replace('Circle K - 76 Stn (Ignacio)', 'Ignacio Gas/76 Stn', regex=False)
profits['sold_to_name'] = profits['sold_to_name'].str.replace('Bartletts General Store', 'Bartletts Market')
profits['sold_to_name'] = profits['sold_to_name'].str.replace('Berk Bowl E - Asian Dept', 'Berkeley Bowl')
profits['sold_to_name'] = profits['sold_to_name'].str.replace('Berk Bowl W - Asian Dept', 'Berkeley Bowl West')
profits['sold_to_name'] = profits['sold_to_name'].str.replace("Jules Thin Crust Oakland", "Jules Thin Crust-Oakland")
profits['sold_to_name'] = profits['sold_to_name'].str.replace("Oliver's-Windsor", "Oliver's- Windsor")
profits['sold_to_name'] = profits['sold_to_name'].str.replace("Perry's-Inverness", "Inverness Park Market")
profits['sold_to_name'] = profits['sold_to_name'].str.replace("Safeway #2456-NEW Petaluma", "Safeway #2456-Son Mtn Pkwy")
profits['sold_to_name'] = profits['sold_to_name'].str.replace("Safeway #2457-NEW SR College", "Safeway #2457-W. College Ave")
profits['sold_to_name'] = profits['sold_to_name'].str.replace("Safeway #2457-W. College Ave.", "Safeway #2457-W. College Ave", regex=False)
profits['sold_to_name'] = profits['sold_to_name'].str.replace("Safeway #2828-Novato", "Safeway #2828-Ignacio")
profits['sold_to_name'] = profits['sold_to_name'].str.replace("Safeway#3011-Petaluma", "Safeway#3011-S. McDowell")
profits['sold_to_name'] = profits['sold_to_name'].str.replace("Target-Sausalito", "Target-Marin City")
profits['sold_to_name'] = profits['sold_to_name'].str.replace("United Liquors-Larkspur", "United Liq/Gather Wine-Lrkspr")
profits['sold_to_name'] = profits['sold_to_name'].str.replace("Walia's Valero", "Walia's Fuel")
profits['sold_to_name'] = profits['sold_to_name'].str.replace("Mollie Stone-San Francisco", "Mollie Stone #8-Tower Mkt.")

In [408]:
profits['sold_to_name'] = profits['sold_to_name'].map(lambda x: re.sub("Ernie's\Z", "Ernie's Tin Bar", x))

In [409]:
stores_to_drop = [
    "Craig Robinson",
    "Club Costa Vida/WFSH",
    "Bill's Liquor",
    "Dan's Market",
    "First Community Bank",
    "Haight & Victoria",
    "Kim's Baskets",
    "Oliver's Market Business Offic",
    "Oliver's Warehouse",
    "Rialto Cinemas Elmwood",
    "Wine & Food Spoken Here",
    "Wine & Foods Spoken Here",
]

In [410]:
profits = profits[profits['sold_to_name'].isin(stores_to_drop) == False]

In [411]:
df_profits = pd.merge(
    profits, 
    store_coords, 
    left_on='sold_to_name',
    right_on='Store',
    how='left'
)

df_profits.head()

,sold_to_name,product_line,quantity,gross_profit,territory,date,Store,Address,City,State,Full Address,lat,lon
0,Farmer Joe's - Fruitvale Ave.,DCD,1941.0,2006.37,EB,2017-01-01,Farmer Joe's - Fruitvale Ave.,3420 Fruitvale Ave.,Oakland,California,"3420 Fruitvale Ave., Oakland, California",37.799477,-122.216227
1,"Seven Eleven-Redwood Hwy,M.V.",DCD,120.0,37.92,MR,2017-01-01,"Seven Eleven-Redwood Hwy,M.V.",650 Redwood Hwy.,Mill Valley,California,"650 Redwood Hwy., Mill Valley, California",37.891146,-122.516022
2,Nugget - #15 Corte Madera,DCD,144.0,32.85,MR,2017-01-01,Nugget - #15 Corte Madera,5627 Paradise Drive,Corte Madera,California,"5627 Paradise Drive, Corte Madera, California",37.919095,-122.507826
3,Nugget - #15 Corte Madera,DCD,57.0,45.41,MR,2017-01-01,Nugget - #15 Corte Madera,5627 Paradise Drive,Corte Madera,California,"5627 Paradise Drive, Corte Madera, California",37.919095,-122.507826
4,Nugget - #16 Novato,DCD,165.0,53.76,MR,2017-01-01,Nugget - #16 Novato,470 Ignacio Blvd.,Novato,California,"470 Ignacio Blvd., Novato, California",38.067484,-122.542898


In [412]:
df_profits.drop_duplicates(inplace=True)

In [413]:
len(profits), len(df_profits)

(27521, 27527)

In [414]:
df_profits.isna().sum()

sold_to_name    0
product_line    0
quantity        0
gross_profit    0
territory       0
date            0
Store           0
Address         0
City            0
State           0
Full Address    0
lat             0
lon             0
dtype: int64

In [416]:
df_profits.to_csv('../data/profits_locations.csv', index=False)